In [ ]:
def plot_layer_thickness(d, materials, ax, fname='final_layer_plot_LTM.png', insert_layer_num=-1, last_insert_layer_thickness=None):
    assert d.shape == materials.shape
    d_plot = 0
    layer_number = d.shape[0]
    # scale x label to optical thickness
    # d = d * (get_n(750., materials)[1:-1, 0])
    for i in range(layer_number):
        # 以600nm的折射率表示
        n_plot = get_n(750, materials)[i + 1].real
        n_last_plot = get_n(750, materials)[i].real
        if i == insert_layer_num:
            ax.plot([d_plot, d_plot + d[i]], [n_plot, n_plot], color='red',
                     label=f'thickness={last_insert_layer_thickness}')
        else:
            ax.plot([d_plot, d_plot + d[i]], [n_plot, n_plot], color='steelblue')
        if i != 0:
            # if n_plot > n_last_plot:
            #     ax.plot([d_plot, d_plot], [n_plot, n_last_plot], c=cmap(f(d[i])), alpha=0.2)
            # else:
            #     ax.plot([d_plot, d_plot], [n_plot, n_last_plot], c=cmap(f(d[i - 1])), alpha=0.2)
            ax.plot([d_plot, d_plot], [n_plot, n_last_plot], c='steelblue')
        d_plot += d[i]
    ax.legend()
    # ax.set_title(f'layer thickness, {layer_number} layers')
    ax.set_xlim(0, 20000)
    # ax.set_xlabel('layer optical thickness')
    # ax.set_ylabel('refractive index at wl=750nm')
    return
